## Load Module Files from GitHub

In [1]:
!rm /content/*.py*

rm: cannot remove '/content/*.py*': No such file or directory


In [2]:
queries = []

filenames = [
             "Conv2DMHAUnit.py",
             "Conv2DmhaLSTMCell.py",
             "conv2d_mha_utils.py"
]

for fn in filenames:
  queries.append(f"https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/{fn}")

filenames_2 = [
             "GraphAttentionHead.py",
             "MultiHeadGraphAttention.py",
             "MultiHeadGraphAttentionLSTMCell.py",
             "mhga_utils.py"
]

for fn in filenames_2:
  queries.append(f"https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/{fn}")

filenames_3 = [
             "MultiHeadAttentionLSTMCell.py",
             "att_bott_utils.py"
]

for fn in filenames_3:
  queries.append(f"https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/AttentionBottleneckLSTM/{fn}")

queries.append("https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/imports.py")

In [3]:
queries

['https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/Conv2DMHAUnit.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/Conv2DmhaLSTMCell.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/conv2d_mha_utils.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/GraphAttentionHead.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/MultiHeadGraphAttention.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/MultiHeadGraphAttentionLSTMCell.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/mhga_utils.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxIn

In [4]:
import subprocess
# Download and import my module files from GitHub
result = subprocess.run(["wget", "--no-cache", "--backups=1"] + queries, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
print(result.stderr.decode("utf-8"))

--2022-07-29 02:53:41--  https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/Conv2DMHAUnit.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3980 (3.9K) [text/plain]
Failed to rename Conv2DMHAUnit.py to Conv2DMHAUnit.py.1: (2) No such file or directory
Saving to: ‘Conv2DMHAUnit.py’

     0K ...                                                   100% 55.6M=0s

2022-07-29 02:53:41 (55.6 MB/s) - ‘Conv2DMHAUnit.py’ saved [3980/3980]

--2022-07-29 02:53:41--  https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/Conv2DAttentionLSTM/Conv2DmhaLSTMCell.py
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 2628 (2.6K)

## Load and Create Google Drive Linked Environment

In [5]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
flight_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/flight_data/aggregated_flight_map_data.csv"
unscaled_maps_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/covid_map_data/infection_maps_unscaled.npy"
unscaled_metadata_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/covid_map_data/infection_unscaled_metadata.json"
scaled_maps_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/covid_map_data/infection_maps_scaled.npy"
scaled_metadata_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/covid_map_data/infection_scaled_metadata.json"
dataset_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/US_time_series_covid_cases.csv"

## Load Sequential Data from Google Drive

In [7]:
from att_bott_utils import load_sequential_data, tf

In [8]:
NUM_DAYS_PER_SAMPLE = 7
IMAGE_X, IMAGE_Y = 64, 64

In [9]:
(X_indices, y_infection, y_death, X_maps), (formatted_X, formatted_adj_mat, formatted_y_infection, formatted_y_death) = load_sequential_data(
    maps_path = unscaled_maps_path,
    metadata_path = unscaled_metadata_path,
    covid_data_path = dataset_path,
    flight_data_path = flight_path,
    image_x = IMAGE_X,
    image_y = IMAGE_Y,
    num_days_per_sample = NUM_DAYS_PER_SAMPLE
)


Generating Sequential Data...
Loading Sequential Graph Data...


100%|██████████| 793/793 [00:21<00:00, 36.78it/s]


Loading Image Indices...


41993it [00:02, 17541.99it/s]


## Scale and Split Train Test

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# MinMax Scale by column
X_max = formatted_X.reshape(-1, formatted_X.shape[-1]).max(axis = 0)
X_min = formatted_X.reshape(-1, formatted_X.shape[-1]).min(axis = 0)
X_nodes_scaled = (formatted_X - X_min) / (X_max - X_min)

mat_max = formatted_adj_mat.reshape(-1,).max(axis = 0)
mat_min = formatted_adj_mat.reshape(-1,).min(axis = 0)
edges_scaled = (formatted_adj_mat - mat_min) / (mat_max - mat_min)

X_nodes_scaled.shape, edges_scaled.shape

((793, 7, 49, 5), (793, 7, 49, 49))

In [16]:
(X_nodes_train, X_nodes_test,
X_edges_train, X_edges_test,
X_map_ind_train, X_map_ind_test,
y_train, y_test) = train_test_split(
    X_nodes_scaled,
    edges_scaled,
    X_indices,
    y_infection,
    train_size = .8,
    random_state = 0
)

In [17]:
def create_flow(
	# global
	batch_size,
	# images
	X_indices,
	raw_X,
	# maps
	X_nodes,
	X_edges,
	# target
	y):
	index = 0
	while True:
		X_image_sample = []
		X_node_sample = []
		X_edge_sample = []
		y_sample = []
		for _ in range(batch_size):
			X_image_sample.append(raw_X[X_indices[index, ...]])
			X_node_sample.append(X_nodes[index, ...])
			X_edge_sample.append(X_edges[index, ...])
			y_sample.append(y[index, ...])
			index += 1
			if index >= X_indices.shape[0]:
				index = 0
				p = np.random.permutation(y.shape[0])
				X_indices = X_indices[p]
				X_nodes = X_nodes[p]
				X_edges = X_edges[p]
				y = y[p]

		yield ((
			np.float32(X_image_sample),
			np.float32(X_node_sample).reshape(batch_size, NUM_DAYS_PER_SAMPLE, -1)
			),
			np.float32(y_sample))

In [18]:
BATCH_SIZE = 16

In [24]:
example_gen = create_flow(
    BATCH_SIZE,
    X_map_ind_train,
    X_maps,
    X_nodes_train,
    X_edges_train,
    y_train
)

In [25]:
import numpy as np

In [26]:
(a, b), c = next(example_gen)

In [27]:
a.shape

(16, 7, 64, 64, 1)

In [28]:
b.shape

(16, 7, 245)

## Create LSTM Model

In [29]:
MODEL_ARCHITECTURE = (16, 16, 16)
SEQUENCE_LENGTH = 49
OUTPUT_SIZE = 49

In [30]:
seq = tf.keras.models.Sequential()
seq.add(tf.keras.layers.Input(shape = (NUM_DAYS_PER_SAMPLE, SEQUENCE_LENGTH * 5)))
for i in range(len(MODEL_ARCHITECTURE) - 1):
  seq.add(tf.keras.layers.LSTM(MODEL_ARCHITECTURE[i], return_sequences = True))
  seq.add(tf.keras.layers.LeakyReLU())
  seq.add(tf.keras.layers.LayerNormalization())
seq.add(tf.keras.layers.LSTM(MODEL_ARCHITECTURE[-1], return_sequences = False))
seq.add(tf.keras.layers.LeakyReLU())
seq.add(tf.keras.layers.LayerNormalization())
# seq.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation = "linear"))

In [31]:
seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 16)             16768     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 7, 16)             0         
                                                                 
 layer_normalization (LayerN  (None, 7, 16)            32        
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 7, 16)             2112      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 16)             0         
                                                                 
 layer_normalization_1 (Laye  (None, 7, 16)            32        
 rNormalization)                                        

## Create ConvLSTM Model

In [32]:
MODEL_ARCHITECTURE = (16, 16, 16)
KERNEL_SIZE = (4, 4)
OUTPUT_SIZE = 49

In [33]:
conv_seq = tf.keras.models.Sequential()
conv_seq.add(tf.keras.layers.Input(shape = (NUM_DAYS_PER_SAMPLE, IMAGE_X, IMAGE_Y, 1)))
for i in range(len(MODEL_ARCHITECTURE) - 1):
  conv_seq.add(tf.keras.layers.ConvLSTM2D(MODEL_ARCHITECTURE[i], KERNEL_SIZE, padding = "same", return_sequences = True))
  conv_seq.add(tf.keras.layers.MaxPool3D((1,4,4)))
  conv_seq.add(tf.keras.layers.LeakyReLU())
  conv_seq.add(tf.keras.layers.LayerNormalization())
conv_seq.add(tf.keras.layers.ConvLSTM2D(MODEL_ARCHITECTURE[-1], KERNEL_SIZE, padding = "same", return_sequences = False))
conv_seq.add(tf.keras.layers.LeakyReLU())
conv_seq.add(tf.keras.layers.LayerNormalization())
conv_seq.add(tf.keras.layers.Flatten())
# conv_seq.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation = "linear"))

In [34]:
conv_seq.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 7, 64, 64, 16)     17472     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 7, 16, 16, 16)    0         
 )                                                               
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 7, 16, 16, 16)     0         
                                                                 
 layer_normalization_3 (Laye  (None, 7, 16, 16, 16)    32        
 rNormalization)                                                 
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 7, 16, 16, 16)     32832     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 7, 4, 4, 16)     

## Create Combination Model

In [35]:
def create_combination_model():
  image_input = tf.keras.layers.Input(shape = (NUM_DAYS_PER_SAMPLE, IMAGE_X, IMAGE_Y, 1))
  seq_input = tf.keras.layers.Input(shape = (NUM_DAYS_PER_SAMPLE, SEQUENCE_LENGTH * 5))

  image_output = conv_seq(image_input)
  seq_output = seq(seq_input)

  concated_output = tf.concat([image_output, seq_output], axis = -1)
  dense_output = tf.keras.layers.Dense(OUTPUT_SIZE, activation = "linear")(concated_output)
  return tf.keras.models.Model(inputs = [image_input, seq_input], outputs = [dense_output])

In [36]:
combination_model = create_combination_model()

In [37]:
combination_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 7, 64, 64,   0           []                               
                                1)]                                                               
                                                                                                  
 input_4 (InputLayer)           [(None, 7, 245)]     0           []                               
                                                                                                  
 sequential_1 (Sequential)      (None, 256)          83232       ['input_3[0][0]']                
                                                                                                  
 sequential (Sequential)        (None, 16)           21088       ['input_4[0][0]']            

## Train Model

In [38]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)

@tf.function
def loss_func(y_true, y_pred):
  squared_diff = tf.square(y_true - y_pred)
  mse = tf.reduce_mean(squared_diff, axis=-1)
  squared_diff_mean = tf.square(tf.reduce_mean(y_true, axis = -1) - tf.reduce_mean(y_pred, axis = -1))
  return mse + squared_diff_mean

combination_model.compile(
    loss = loss_func,
    optimizer = opt,
    metrics = ["mse", "mae"]
)

In [39]:
BATCH_SIZE = 16
EPOCHS = 20

In [40]:
train_generator = create_flow(
    BATCH_SIZE,
    X_map_ind_train,
    X_maps,
    X_nodes_train,
    X_edges_train,
    y_train
)

In [41]:
STEPS_PER_EPOCH = X_map_ind_train.shape[0] // BATCH_SIZE

In [42]:
import numpy as np

In [43]:
history = combination_model.fit(
    x = train_generator,
    batch_size = BATCH_SIZE,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS
    )

Epoch 1/20
39/39 [==============================] - 28s 162ms/step - loss: 0.1225 - mse: 0.1177 - mae: 0.1844
Epoch 2/20
39/39 [==============================] - 6s 165ms/step - loss: 7.5427e-04 - mse: 7.5026e-04 - mae: 0.0200
Epoch 3/20
39/39 [==============================] - 6s 166ms/step - loss: 1.1015e-04 - mse: 1.0971e-04 - mae: 0.0078
Epoch 4/20
39/39 [==============================] - 6s 165ms/step - loss: 6.2673e-05 - mse: 6.2410e-05 - mae: 0.0059
Epoch 5/20
39/39 [==============================] - 6s 163ms/step - loss: 4.5728e-05 - mse: 4.5497e-05 - mae: 0.0051
Epoch 6/20
39/39 [==============================] - 6s 162ms/step - loss: 3.8252e-05 - mse: 3.8033e-05 - mae: 0.0047
Epoch 7/20
39/39 [==============================] - 6s 163ms/step - loss: 3.1988e-05 - mse: 3.1828e-05 - mae: 0.0043
Epoch 8/20
39/39 [==============================] - 6s 161ms/step - loss: 2.6901e-05 - mse: 2.6702e-05 - mae: 0.0040
Epoch 9/20
39/39 [==============================] - 6s 164ms/step - los

In [45]:
test_generator = create_flow(
    batch_size = BATCH_SIZE,
    X_indices = X_map_ind_test,
    raw_X = X_maps,
    X_nodes = X_nodes_test,
    X_edges = X_edges_test,
    y = y_test
)

In [46]:
STEPS = X_map_ind_test.shape[0] // BATCH_SIZE

In [48]:
eval_res = combination_model.evaluate(
    x = test_generator,
    steps = STEPS,
    batch_size = BATCH_SIZE)

9/9 [==============================] - 2s 32ms/step - loss: 9.2354e-06 - mse: 9.1074e-06 - mae: 0.0024


In [49]:
eval_res

[9.235418474418111e-06, 9.107360710913781e-06, 0.002375022741034627]

In [50]:
combination_model.save_weights("/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/combination_weights/model_new_metric/model_weights")